In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [10]:
df = pd.read_csv('titanic_cleand.csv')

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Survived'])
y = df["Survived"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)